<font color = "purple", size = 4> The ArtMIND algorithm for the two galleries our team is assigned for final judging, Gallery 155 and Gallery 161

In [1]:
import pandas as pd

In [2]:
#read the file of manually generated social tags.
df = pd.read_excel("socialtags.xlsx")

In [3]:
df.head(3)

,ObjectID,Social Tags (Primary),Title
0,59205,"landscape, water, trees, stone, sky, building,...",Bay of L'Estaque
1,59197,"portrait, boy, hat, blue",Boy with a Toy Soldier (Portrait of Jean de La...
2,59213,"person, man, hat, black",Le Bon Bock


In [4]:
listoftags = [df["Social Tags (Primary)"].iloc[ind].split(", ") for ind in range(len(df))] 

In [5]:
df["SocialTags"] = listoftags
df.head(3)

,ObjectID,Social Tags (Primary),Title,SocialTags
0,59205,"landscape, water, trees, stone, sky, building,...",Bay of L'Estaque,"[landscape, water, trees, stone, sky, building..."
1,59197,"portrait, boy, hat, blue",Boy with a Toy Soldier (Portrait of Jean de La...,"[portrait, boy, hat, blue]"
2,59213,"person, man, hat, black",Le Bon Bock,"[person, man, hat, black]"


In [6]:
df2 = df.drop("Social Tags (Primary)", axis = 1, inplace = False)

In [7]:
tags = [item for sublist in listoftags for item in sublist]
len(tags)

247

In [8]:
#the unique social tags 
tags = set(tags)
tags

{' white',
 'animal',
 'black',
 'blue',
 'boat',
 'boy',
 'bridge',
 'building',
 'cloth',
 'detailed',
 'flowers',
 'gold',
 'green',
 'hat',
 'household item',
 'landscape',
 'man',
 'nude',
 'people',
 'person',
 'pink',
 'portrait',
 'red',
 'sky',
 'still life',
 'stone',
 'tan',
 'trees',
 'trees,water',
 'vase',
 'water',
 'white',
 'woman'}

<font color = "purple"> call the API for information about the works just in gallery 155 and gallery 161

In [9]:
import numpy as np
import itertools 
import requests

In [11]:
galleries = [155, 161] 

In [12]:
#ids will be filled with the object ids in the two galleries.
ids = []

In [13]:
p = {"api_token": token, 'name': galleries[0]}
req = requests.get(url, p) 
data = req.json()
gal_ids = data["ObjectIDs"]


In [14]:
for name in galleries: 
    p = {"api_token": token, 'name': name}
    req = requests.get(url, p) 
    data = req.json()
    gal_ids = data["ObjectIDs"]
    ids.append(gal_ids) 

In [15]:
flat_ids = [item for sublist in ids for item in sublist]

In [17]:
#retrieve information about the objects in the list ids
frames = []
for objectid in flat_ids : 
    p = {"api_token": token, 'objectId':objectid}
    req = requests.get(url,p) 
    data = req.json()
    id_features = [data["ObjectID"], data["Classification"], data["Style"], data["Geography"], data["Dated"]]
    frames.append(id_features) 

In [18]:
#convert the information into a Pandas data frame, and write as a csv file. 
features = pd.DataFrame(frames, columns = ["ObjectID", "Classification", "Style", "Geography", "Date"])

In [63]:
features.head(2)

,ObjectID,Classification,Style,Geography,Date
0,42505,Paintings,Realist,"Made in Philadelphia, Pennsylvania, United Sta...",1880
1,57411,Sculpture,Realist,"Made in France, Europe",c. 1874


In [64]:
features.to_csv("limited_objectinfo_final2.csv") 

<font color = "purple"> algorithm begins here.

<font color = "purple"> feature preprocessing

In [104]:
#Geography
for ind in range(len(features)) : 
    
    if type(features.Geography.iloc[ind]) == str : 
        
        if ("France" in features.Geography.iloc[ind]) : 
            features.Geography.iloc[ind] = "France"
            
        if ("Italy" in features.Geography.iloc[ind]) : 
            features.Geography.iloc[ind] = "Italy"
            
        if ("England" in features.Geography.iloc[ind]) | ("Great Britain" in features.Geography.iloc[ind]): 
            features.Geography.iloc[ind] = "Great Britain"
            
        if ("Belgium" in features.Geography.iloc[ind]) : 
            features.Geography.iloc[ind] = "Belgium" 
        
        if "Netherlands" in features.Geography.iloc[ind] : 
            features.Geography.iloc[ind] = "Netherlands" 

        if "Pennsylvania" in features.Geography.iloc[ind] : 
            features.Geography.iloc[ind] = "Pennsylvania" 

C:\Users\sowmya\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


<font color = "purple"> algorithm

In [19]:
#the ten works from which we determine the user's preferences
tenworks = pd.read_excel("tenworks.xlsx")
obj = list(tenworks.ObjectID)

In [114]:
#reset index in order to create dummy variables for the social tags.
df3 = df2.set_index("ObjectID")
df3.head(2)

,Title,SocialTags
ObjectID,,
59205,Bay of L'Estaque,"[landscape, water, trees, stone, sky, building..."
59197,Boy with a Toy Soldier (Portrait of Jean de La...,"[portrait, boy, hat, blue]"


In [122]:
s = df3["SocialTags"]
dummytags = pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)
dummytags.head(2)

,white,animal,black,blue,boat,boy,bridge,building,cloth,detailed,...,sky,still life,stone,tan,trees,"trees,water",vase,water,white,woman
ObjectID,,,,,,,,,,,,,,,,,,,,,
59205,0,0,0,0,0,0,0,1,0,0,...,1,0,1,0,1,0,0,1,0,0
59197,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [125]:
#merge the table of social tags with the table containing the other features.
feat = features.merge(dummytags, left_on = "ObjectID", right_index = True, how = 'outer')

In [127]:
feat.drop("SocialTags", axis = 1, inplace = True) 
feat.head(2)

,ObjectID,Classification,Style,Geography,Date,white,animal,black,blue,boat,...,sky,still life,stone,tan,trees,"trees,water",vase,water,white,woman
0,42505,Paintings,Realist,Pennsylvania,1880,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,57411,Sculpture,Realist,France,1874,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [129]:
#create dummy variables for the remaining features.
feat.drop("Date", axis = 1, inplace = True)
feat = pd.get_dummies(feat, columns = ["Classification", "Style", "Geography"])

In [130]:
df_feat = feat.set_index("ObjectID")
df_feat.head(1)

,white,animal,black,blue,boat,boy,bridge,building,cloth,detailed,...,Style_Post-Impressionist,Style_Realist,Style_Romantic,Style_Symbolist,Geography_Belgium,Geography_France,Geography_Great Britain,Geography_Italy,Geography_Netherlands,Geography_Pennsylvania
ObjectID,,,,,,,,,,,,,,,,,,,,,
42505,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1


In [141]:
#function that inputs a list of objects the user is shown and a list of their preferences, 
#and outputs the top 3 recommendations 
#the function makes use of df_feat from above 

def featmat(obj, pref) : 
    #feature_matrix will contain only the ten rows corresponding to the art shown to the user
    feature_matrix = df_feat.loc[obj]

    adapted_matrix = np.zeros((10,np.shape(feature_matrix)[1] )) 

    for i in range(len(pref)) : 
        adapted_matrix[i] = pref[i] * feature_matrix.iloc[i]

    #for each column, find the mean
    feature_means = adapted_matrix.mean(axis = 0) 
 
    norms = []
    for ind in df_feat.index : 
        norms.append({"objid":ind, "norm":np.linalg.norm(np.array(df_feat.loc[ind])-np.array(feature_means), ord = 1)})
    norms_df = pd.DataFrame(norms)
    norms_df.sort_values("norm", axis =0, inplace = True)

    normalized = [1-1/(1+norms_df.norm.iloc[i]) for i in range(len(norms_df))]
    norms_df["normalized"] = normalized 
    norms_df.sort_values("normalized", axis = 0, ascending = False, inplace = True) 

    return(norms_df.head(3))